In [87]:
import os
from dotenv import load_dotenv
import requests
from pytz import timezone
import datetime

load_dotenv()
api_key = os.getenv("API_KEY")


In [107]:
api_url = "https://freeserv.dukascopy.com/2.0/?path=api/instrumentList"

response = requests.get(api_url, params={'key': api_key})

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Access the response data
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")

[{'id': '2000', 'name': 'AUD/JPY', 'pipValue': 0.01}, {'id': '2002', 'name': 'AUD/USD', 'pipValue': 0.0001}, {'id': '2004', 'name': 'CAD/JPY', 'pipValue': 0.01}, {'id': '2006', 'name': 'CHF/JPY', 'pipValue': 0.01}, {'id': '2008', 'name': 'EUR/CHF', 'pipValue': 0.0001}, {'id': '2010', 'name': 'EUR/DKK', 'pipValue': 0.0001}, {'id': '2012', 'name': 'EUR/GBP', 'pipValue': 0.0001}, {'id': '2014', 'name': 'EUR/JPY', 'pipValue': 0.01}, {'id': '2016', 'name': 'EUR/NOK', 'pipValue': 0.0001}, {'id': '2018', 'name': 'EUR/SEK', 'pipValue': 0.0001}, {'id': '2020', 'name': 'EUR/USD', 'pipValue': 0.0001}, {'id': '2022', 'name': 'GBP/CHF', 'pipValue': 0.0001}, {'id': '2024', 'name': 'GBP/JPY', 'pipValue': 0.01}, {'id': '2026', 'name': 'GBP/USD', 'pipValue': 0.0001}, {'id': '2028', 'name': 'USD/CAD', 'pipValue': 0.0001}, {'id': '2030', 'name': 'USD/CHF', 'pipValue': 0.0001}, {'id': '2032', 'name': 'USD/JPY', 'pipValue': 0.01}, {'id': '2034', 'name': 'NZD/USD', 'pipValue': 0.0001}, {'id': '2036', 'name'

In [46]:
api_url = "https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices"

instrument = "2024"
timeFrame = "1min"

response = requests.get(api_url, params={'key': api_key,
                                         'instrument': instrument,
                                         'timeFrame': timeFrame})

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Access the response data
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")

{'id': '2024', 'timeFrame': '1min', 'offerSide': 'bid', 'candles': [{'timestamp': 1684208640000, 'bid_open': 170.424, 'bid_high': 170.426, 'bid_low': 170.42, 'bid_close': 170.425, 'bid_volume': 24.82}, {'timestamp': 1684208700000, 'bid_open': 170.423, 'bid_high': 170.423, 'bid_low': 170.391, 'bid_close': 170.4, 'bid_volume': 64.03}, {'timestamp': 1684208760000, 'bid_open': 170.401, 'bid_high': 170.402, 'bid_low': 170.395, 'bid_close': 170.402, 'bid_volume': 28.73}, {'timestamp': 1684208820000, 'bid_open': 170.403, 'bid_high': 170.403, 'bid_low': 170.386, 'bid_close': 170.386, 'bid_volume': 55.67}, {'timestamp': 1684208880000, 'bid_open': 170.387, 'bid_high': 170.387, 'bid_low': 170.361, 'bid_close': 170.362, 'bid_volume': 57.38}, {'timestamp': 1684208940000, 'bid_open': 170.361, 'bid_high': 170.361, 'bid_low': 170.336, 'bid_close': 170.344, 'bid_volume': 67.99}, {'timestamp': 1684209000000, 'bid_open': 170.343, 'bid_high': 170.357, 'bid_low': 170.343, 'bid_close': 170.346, 'bid_volume'

In [102]:
dt = datetime.datetime(2010, 1, 1, 0, 0, 0)
start_timestamp = int(dt.timestamp()*1000)

current_datetime = datetime.datetime.now()
end_timestamp = int(current_datetime.timestamp()*1000)

candles_per_request = 5000       # Number of candles to retrieve per request

# Initialize an empty list to store the retrieved data
data = []

# Calculate the number of requests needed based on the total number of candles
total_candles = (end_timestamp - start_timestamp) // (60 * 1000)  # Convert milliseconds to minutes
total_requests = (total_candles // candles_per_request) + 1

print(total_requests)


1407


In [103]:
# Loop through each request
for request_number in range(total_requests):
    print(request_number)
    # Calculate the start and end timestamps for each request
    request_start = start_timestamp + (request_number * candles_per_request * 60 * 1000)  # Convert minutes to milliseconds
    request_end = request_start + (candles_per_request * 60 * 1000) - 1
    
    response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'count': 5000,
                                             'start': request_start,
                                             'end': request_end,})
    
    # Parse the response and extract the candle data
    if response.status_code == 200:
        response_data = response.json()
        candles = response_data['candles']
        
        # Append the retrieved candles to the historical_data list
        data.extend(candles)
    else:
        print(f"Error in API request {request_number + 1}: {response.status_code}")
        break

# Print the condensed historical data
for candle in historical_data:
    print(candle)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [104]:
data

[{'timestamp': 1326245820000,
  'bid_open': 118.906,
  'bid_high': 118.916,
  'bid_low': 118.905,
  'bid_close': 118.916,
  'bid_volume': 51.75},
 {'timestamp': 1326245880000,
  'bid_open': 118.916,
  'bid_high': 118.922,
  'bid_low': 118.908,
  'bid_close': 118.908,
  'bid_volume': 51.64},
 {'timestamp': 1326245940000,
  'bid_open': 118.908,
  'bid_high': 118.908,
  'bid_low': 118.904,
  'bid_close': 118.904,
  'bid_volume': 34.51},
 {'timestamp': 1326246000000,
  'bid_open': 118.904,
  'bid_high': 118.913,
  'bid_low': 118.904,
  'bid_close': 118.912,
  'bid_volume': 64.04},
 {'timestamp': 1326246060000,
  'bid_open': 118.914,
  'bid_high': 118.918,
  'bid_low': 118.914,
  'bid_close': 118.918,
  'bid_volume': 106.5},
 {'timestamp': 1326246120000,
  'bid_open': 118.918,
  'bid_high': 118.921,
  'bid_low': 118.913,
  'bid_close': 118.921,
  'bid_volume': 48.86},
 {'timestamp': 1326246180000,
  'bid_open': 118.921,
  'bid_high': 118.932,
  'bid_low': 118.913,
  'bid_close': 118.932,
  

In [105]:
df = pd.DataFrame(data)

df['datetime'] = pd.to_datetime(df.timestamp, unit='ms')
df['datetime'] = df['datetime'].dt.tz_localize('UTC')
london = timezone('Europe/London')

df['datetime'] = df['datetime'].dt.tz_convert(london)

df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [78]:
df.to_csv('./test.csv')

In [106]:
df

,timestamp,bid_open,bid_high,bid_low,bid_close,bid_volume,datetime
0,1326245820000,118.906,118.916,118.905,118.916,51.75,2012-01-11 01:37:00
1,1326245880000,118.916,118.922,118.908,118.908,51.64,2012-01-11 01:38:00
2,1326245940000,118.908,118.908,118.904,118.904,34.51,2012-01-11 01:39:00
3,1326246000000,118.904,118.913,118.904,118.912,64.04,2012-01-11 01:40:00
4,1326246060000,118.914,118.918,118.914,118.918,106.50,2012-01-11 01:41:00
...,...,...,...,...,...,...,...
4572629,1684241280000,170.216,170.241,170.167,170.167,314.60,2023-05-16 13:48:00
4572630,1684241340000,170.167,170.214,170.156,170.197,260.19,2023-05-16 13:49:00
4572631,1684241400000,170.197,170.244,170.185,170.208,339.36,2023-05-16 13:50:00
4572632,1684241460000,170.207,170.233,170.182,170.228,371.81,2023-05-16 13:51:00


In [43]:
data['candles']

[{'timestamp': 1684207140000,
  'bid_open': 170.368,
  'bid_high': 170.378,
  'bid_low': 170.361,
  'bid_close': 170.372,
  'bid_volume': 91.21},
 {'timestamp': 1684207200000,
  'bid_open': 170.373,
  'bid_high': 170.375,
  'bid_low': 170.363,
  'bid_close': 170.368,
  'bid_volume': 28.92},
 {'timestamp': 1684207260000,
  'bid_open': 170.367,
  'bid_high': 170.367,
  'bid_low': 170.358,
  'bid_close': 170.358,
  'bid_volume': 13.24},
 {'timestamp': 1684207320000,
  'bid_open': 170.358,
  'bid_high': 170.365,
  'bid_low': 170.35,
  'bid_close': 170.364,
  'bid_volume': 41.51},
 {'timestamp': 1684207380000,
  'bid_open': 170.363,
  'bid_high': 170.388,
  'bid_low': 170.363,
  'bid_close': 170.385,
  'bid_volume': 24},
 {'timestamp': 1684207440000,
  'bid_open': 170.387,
  'bid_high': 170.391,
  'bid_low': 170.384,
  'bid_close': 170.385,
  'bid_volume': 21.69},
 {'timestamp': 1684207500000,
  'bid_open': 170.382,
  'bid_high': 170.391,
  'bid_low': 170.377,
  'bid_close': 170.39,
  'bid_

TypeError: list indices must be integers or slices, not str

In [41]:
5000/60/24

3.472222222222222

In [45]:

start_timestamp = 1678924800  # Example start timestamp
end_timestamp = 1681599600   # Example end timestamp
candles_per_request = 5000       # Number of candles to retrieve per request

# Initialize an empty list to store the retrieved data
historical_data = []

# Calculate the number of requests needed based on the total number of candles
total_candles = end_timestamp - start_timestamp
total_requests = (total_candles // candles_per_request) + 1

# Loop through each request
for request_number in range(total_requests):
    # Calculate the start and end timestamps for each request
    request_start = start_timestamp + (request_number * candles_per_request)
    request_end = request_start + candles_per_request - 1
    
    response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'start': request_start,
                                             'end': request_end,})
    
    # Parse the response and extract the candle data
    if response.status_code == 200:
        response_data = response.json()
        candles = response_data['candles']
        
        # Append the retrieved candles to the historical_data list
        historical_data.extend(candles)
    else:
        print(f"Error in API request {request_number + 1}: {response.status_code}")
        break

# Print the condensed historical data
for candle in historical_data:
    print(candle)

Error in API request 1: 204


In [114]:
api_url = "https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices"

response = requests.get(api_url, params={'key': api_key,
                                            'instrument': instrument,
                                            'timeFrame': timeFrame,
                                            'count': 5000,
                                            'start': start_timestamp})


In [127]:
dt = datetime.datetime(2010, 1, 1, 0, 0, 0)
start_timestamp = int(dt.timestamp()*1000)

In [128]:

# Parse the response and extract the candle data
if response.status_code == 200:
    response_data = response.json()
    candles = response_data['candles']
else:
    print(f"Error in API request {request_number + 1}: {response.status_code}")


In [129]:
candles

[{'timestamp': 1683768180000,
  'bid_open': 169.42,
  'bid_high': 169.443,
  'bid_low': 169.412,
  'bid_close': 169.436,
  'bid_volume': 67.9},
 {'timestamp': 1683768240000,
  'bid_open': 169.436,
  'bid_high': 169.443,
  'bid_low': 169.409,
  'bid_close': 169.419,
  'bid_volume': 115.12},
 {'timestamp': 1683768300000,
  'bid_open': 169.419,
  'bid_high': 169.419,
  'bid_low': 169.379,
  'bid_close': 169.384,
  'bid_volume': 129.74},
 {'timestamp': 1683768360000,
  'bid_open': 169.386,
  'bid_high': 169.397,
  'bid_low': 169.38,
  'bid_close': 169.39,
  'bid_volume': 57.36},
 {'timestamp': 1683768420000,
  'bid_open': 169.39,
  'bid_high': 169.414,
  'bid_low': 169.386,
  'bid_close': 169.407,
  'bid_volume': 61.99},
 {'timestamp': 1683768480000,
  'bid_open': 169.408,
  'bid_high': 169.439,
  'bid_low': 169.404,
  'bid_close': 169.439,
  'bid_volume': 39.72},
 {'timestamp': 1683768540000,
  'bid_open': 169.442,
  'bid_high': 169.443,
  'bid_low': 169.428,
  'bid_close': 169.437,
  'bi